# Watson Visual Recognition Training with Spectrogram Images from SETI Signal Data

* https://www.ibm.com/watson/developercloud/visual-recognition/api/v3/
* https://www.ibm.com/watson/developercloud/doc/visual-recognition/customizing.html
* https://github.com/watson-developer-cloud/python-sdk
* https://github.com/watson-developer-cloud/python-sdk/blob/master/watson_developer_cloud/visual_recognition_v3.py

<hr>

## Install the Watson Developer Cloud Python SDK

* Install the Python SDK if has not been previously installed
* Restart the kernel, after installing the SDK

In [ ]:
#!pip install --user --upgrade watson-developer-cloud

In [ ]:
from __future__ import division


import glob
import json
import os
import time
import timeit
import zipfile

from random import randint

import numpy as np


import re



In [ ]:
teamname = "Machine_Contact_Enterprise_SETI"
zipfilefolder = os.path.join( os.environ['PWD'], teamname ) + '/zippedPngsTestDay1'


mydatafolder = os.path.join( os.environ['PWD'], teamname )  
if os.path.exists(mydatafolder) is False:
    raise Exception('The data folder does not exist')

if os.path.exists(zipfilefolder) is False:
    raise Exception('The zip folder does not exist')

In [ ]:
from watson_developer_cloud import VisualRecognitionV3
apiVer = VisualRecognitionV3.latest_version #'2016-05-20'
classifier_prefix = teamname + "_v1_"

#You can sign up with WatsonVR through Bluemix to get a key
#However, Hackathon participants will be provided with a WATSON VR key that has more free API calls per day.
apiKey = 'c70486e04c0e6ac12314da71f2fdb88992420c33'  


In [ ]:
vr = VisualRecognitionV3(apiVer, api_key=apiKey)

<br/>
## Look For Existing Custom Classifier
Use an existing custom classifier (and update) if one exists, else a new custom classifier will be created

In [ ]:
classifier_id = None
classifier = None

classifiers = vr.list_classifiers()

for c in classifiers['classifiers']:
    if c['status'] == 'ready' and (classifier_prefix in c['classifier_id']):
        classifier_id = c['classifier_id']


if classifier_id is not None:
    classifier = vr.get_classifier(classifier_id)
    print '\r\nFound classifer:\r\n\r\n{}'.format(json.dumps(classifier, indent=2))
else:
    print 'No custom classifier available\r\n'
    print(json.dumps(classifiers, indent=2))

<br/>
## Send the Images Archives to the Watson Visual Recognition Service for Training

* https://www.ibm.com/watson/developercloud/doc/visual-recognition/customizing.html
* https://www.ibm.com/watson/developercloud/visual-recognition/api/v3/
* https://github.com/watson-developer-cloud/python-sdk

In [ ]:
squiggle = sorted(glob.glob('{}/classification_*_squiggle.zip'.format(zipfilefolder)))
narrowband = sorted(glob.glob('{}/classification_*_narrowband.zip'.format(zipfilefolder)))
narrowbanddrd = sorted(glob.glob('{}/classification_*_narrowbanddrd.zip'.format(zipfilefolder)))
noise = sorted(glob.glob('{}/classification_*_noise.zip'.format(zipfilefolder)))
#brightpixel = sorted(glob.glob('{}/classification_*_brightpixel.zip'.format(zipfilefolder)))
#squarepulsednarrowband = sorted(glob.glob('{}/classification_*_squarepulsednarrowband.zip'.format(zipfilefolder)))
#squigglesquarepulsednarrowband = sorted(glob.glob('{}/classification_*_squigglesquarepulsednarrowband.zip'.format(zipfilefolder)))

sq = len(squiggle)
nb = len(narrowband)
nd = len(narrowbanddrd)
ns = len(noise)
#nbp = len(brightpixel)
#nsp = len(squarepulsednarrowband)
#nss = len(squigglesquarepulsednarrowband)

## Possible todo here: Try using the 'noise' as a "negative" example when training Watson. See the Watson documentation.

## for prototyping. 
num = 2
#num = max(sq, nb, nd, ns)


if classifier_id is None:
    print 'Adding custom classifier ... this may take awhile'
else:
    print 'Updating custom classifier {} ... this may take awhile'.format(classifier_id)

for i in range(num):
    squiggle_p = open(squiggle[i], 'rb') if i < sq else None
    narrowband_p = open(narrowband[i], 'rb') if i < nb else None
    narrowbanddrd_p = open(narrowbanddrd[i], 'rb') if i < nd else None
    noise_p = open(noise[i], 'rb') if i < ns else None
    #brightpixel_p = open(brightpixel[i], 'rb') if i < nb else None
    #squarepulsednarrowband_p = open(squarepulsednarrowband[i], 'rb') if i < nd else None
    #squigglesquarepulsednarrowband_p = open(squigglesquarepulsednarrowband[i], 'rb') if i < ns else None
    

    if classifier_id is None:
        print 'Creating new classifier. Batch {}'.format(i)
        classifier = vr.create_classifier(
            classifier_prefix,
            squiggle_positive_examples = squiggle_p,
            narrowband_positive_examples = narrowband_p,
            narrowbanddrd_positive_examples = narrowbanddrd_p,
            noise_positive_examples = noise_p
            #brightpixel_positive_examples = brightpixel_p,
            #squarepulsednarrowband_positive_examples = squarepulsednarrowband_p,
            #squigglesquarepulsednarrowband_positive_examples = squigglesquarepulsednarrowband_p
        )
        
        classifier_id = classifier['classifier_id']
    else:
        print 'Updating classifier. Batch {}'.format(i)
        classifier = vr.update_classifier(
            classifier_id,
            squiggle_positive_examples = squiggle_p,
            narrowband_positive_examples = narrowband_p,
            narrowbanddrd_positive_examples = narrowbanddrd_p,
            noise_positive_examples = noise_p
            #brightpixel_positive_examples = brightpixel_p,
            #squarepulsednarrowband_positive_examples = squarepulsednarrowband_p,
            #squigglesquarepulsednarrowband_positive_examples = squigglesquarepulsednarrowband_p 
        )

    if squiggle_p is not None:
        squiggle_p.close()
    if narrowband_p is not None:
        narrowband_p.close()
    if narrowbanddrd_p is not None:
        narrowbanddrd_p.close()
    if noise_p is not None:
        noise_p.close()
    #if brightpixel_p is not None:
    #    brightpixel_p.close()
    #if squarepulsednarrowband_p is not None:
    #    squarepulsednarrowband_p.close()
    #if squigglesquarepulsednarrowband_p is not None:
    #    squigglesquarepulsednarrowband_p.close()

    if classifier is not None:
        print('Classifier: {}'.format(classifier_id))
        status = classifier['status']
        startTimer = timeit.default_timer()
        print 'Working ...'
        while status in ['training', 'retraining']:
            # print('Status: {}'.format(status))
            time.sleep(10)
            classifier = vr.get_classifier(classifier_id)
            status = classifier['status']
        stopTimer = timeit.default_timer()
        print '{} took {} minutes'.format('Training' if i == 0 else 'Retraining', int(stopTimer - startTimer) / 60)

# print(json.dumps(vr.get_classifier(classifier_id), indent=2))


In [ ]:
for image_archive_name in test_set:
    image_count = 0
    # count number of images in <image_archive_name>
    with zipfile.ZipFile(image_archive_name,'r') as image_archive:
        images = image_archive.namelist()
        image_count = len(images)

  
    # bulk classify images in <image_archive_name>
    with open(image_archive_name, 'rb') as images_file:
        print 'Running test ({} images) for {}... this may take a while.'.format(image_count, image_archive_name)
        classify_results = vr.classify(images_file=images_file, classifier_ids=[classifier_id], threshold=0.0)
        # print(json.dumps(classify_results, indent=2))
        # identify class from ZIP file name, e.g. testset_10_squiggle.zip
        mo = re.match('^(.+)_(\d+)_(.+)\.zip$',image_archive_name.split('/')[-1])
        classification = mo.group(3)
        resdict = results_group_by_class[classification]
        passed = 0

        for classify_result in classify_results['images']:
            pngfilename = classify_result['image'].split('/')[-1]
            uuid = pngfilename.split('.')[0]

            maxscore = 0
            maxscoreclass = None

            if "error" in classify_result:
                # print error information
                print classify_result
                #add to failed list
                failed_to_classify_uuid_list.append(uuid)                  
            else:
                classifiers_arr = classify_result['classifiers']
                score_list = []
                for classifier_result in classifiers_arr:
                    for class_result in classifier_result['classes']:
                        score_list.append((class_result['class'],class_result['score']))
                        if class_result['score'] > maxscore:
                            maxscore = class_result['score']
                            maxscoreclass = class_result['class']

                #sort alphabetically
                score_list.sort(key = lambda x: x[0])
                score_list = map(lambda x:x[1], score_list)

                if maxscoreclass is None:
                    print 'Failed: {} - Actual: {}, No classification returned'.format(pngfilename, classification)
                    #print(json.dumps(classify_result, indent=2))
                elif maxscoreclass != classification:
                    print 'Failed: {} - Actual: {}, Watson Predicted: {} ({})'.format(pngfilename, classification, maxscoreclass, maxscore)
                else:
                    passed += 1
                    print 'Passed: {} - Actual: {}, Watson Predicted: {} ({})'.format(pngfilename, classification, maxscoreclass, maxscore)

                if maxscoreclass is not None:
                    resdict['signal_classification'].append(classification)
                    resdict['uuid'].append(uuid)
                    resdict['watson_class'].append(maxscoreclass)
                    resdict['watson_class_score'].append(maxscore)
                    resdict['scores'].append(score_list)
                else:
                    #add to failed list
                    failed_to_classify_uuid_list.append(uuid)
        print 'Test Score: {}% ({} of {} Passed)'.format(int((float(passed) / image_count) * 100), passed, image_count)
        print 'Tested {} images in {} minutes'.format(image_count, int(stopTimer - startTimer) / 60)

In [ ]:
import sys

fwriter = csv.writer(sys.stdout, delimiter=',')
print len(testresults)
for items in testresults:
    fwriter.writerow(items)

In [ ]:
# testresults

In [ ]:
# import pickle
is istrainingset is True:
    pickle.dump(results_group_by_class, open(mydatafolder + '/' + "watson_results2.pickle", "w"))

In [ ]:
#TODO number results
is istrainingset is True:
    watson_results = pickle.load(open(mydatafolder + '/' + "watson_results2.pickle","r"))

    # reorganize the watson_results dictionary to extract
    # a list of [true_class, [scores], estimated_class] and
    # use these for measuring our model's performance

    class_scores = []
    for k in watson_results.keys():
        class_scores += zip(watson_results[k]['uuid'], watson_results[k]['signal_classification'], watson_results[k]['scores'], watson_results[k]['watson_class'] )

In [ ]:
class_scores[50]

In [ ]:
len(class_scores)

In [ ]:
from sklearn.metrics import classification_report
import sklearn

y_train = [x[1] for x in class_scores]
y_pred = [x[3] for x in class_scores]
y_prob = [x[2] for x in class_scores]
#we normalize the Watson score values to 1 in order to use them in the log_loss calculation even though the Watson VR scores are not true class prediction probabilities
y_prob = map(lambda x: (x, sum(x)), y_prob)
y_prob = map(lambda x: [y / x[1] for y in x[0]], y_prob)

print sklearn.metrics.classification_report(y_train,y_pred)
print sklearn.metrics.confusion_matrix(y_train,y_pred)
print("Classification accuracy: %0.6f" % sklearn.metrics.accuracy_score(y_train,y_pred) )
print("Log Loss: %0.6f" % sklearn.metrics.log_loss(y_train,y_prob) )

# Generate CSV file for Scoreboard

#### NOTE: This uses the PNG files created in the Step 3 notebook, which only contain the BASIC4 data set. The code challenge and hackathon will be based on the Primary Data Set which contains 7 signal classes

In [ ]:
import csv
#TODO number scores
my_output_results = mydatafolder + '/' + 'watson_scores2.csv'
with open(my_output_results, 'w') as csvfile:
    fwriter = csv.writer(csvfile, delimiter=',')
    for row in class_scores:
        fwriter.writerow([row[0]] + row[2])

In [ ]:
#print out the first 10 lines just to show it makes sense

!cat $mydatafolder/watson_scores2.csv
